In [6]:
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import os
import shutil 
import xarray as xr
import glob 
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

In [9]:
#%% set up data paths and output directories
#path for python parser script
parserpath = r'W:\LOWE_IMOS_Deakin_Collab_JUN2020\Data\SofarSpotter\SofarParser\parser_v1.12.0'
parser = 'parser_v1.12.0.py'

dpath = r'P:\HANSEN_UWA-UNSW_Linkage\Data\CSIRO_Collab\DEP02_FEB-AUG'

#spotters to process
buoyID = ['SPOT-1002','SPOT-1011','SPOT-1018','SPOT-1026']

def sofar_date_parser(x1,x2,x3,x4,x5,x6,x7):
    t = datetime(int(x1),int(x2),int(x3),int(x4),int(x5),int(x6))+timedelta(seconds=float(x7)/1000)
    return t

for buoy in buoyID: 
    #output path and path to raw data
    outpath = os.path.join(dpath,buoy)
    datapath = os.path.join(outpath,'raw')  
    print('outpath is ' + outpath)
    print('datapath is' + datapath)
    
    #determien number of files and chunks to process
    files = os.listdir(datapath)
    dum = []
    [dum.append(i[0:4]) for i in files]
    file_nums = np.unique(np.array(dum))
    chunk_size= 10
    
    #Create tmp directory for moving files to process
    tmp_path = os.path.join(datapath, 'tmp')
    os.mkdir(tmp_path)
    shutil.copyfile(os.path.join(parserpath, parser), os.path.join(tmp_path,parser))
    
    #process data files
    os.chdir(tmp_path)
    for i in range(0, len(file_nums), chunk_size):
#         print('Processing files ' + str(i) + ' to ' + str(i+chunk_size) + ' out of ' + str(len(file_nums)))
        chunk = file_nums[i:i+chunk_size]
        #move files in chunk to temp folder
        for j in chunk:      
            for k in files:
                if k[0:4]==j:
                    shutil.copyfile(os.path.join(datapath, k), os.path.join(tmp_path, k))
        #now process
        try:
            %run -i parser_v1.12.0.py
            flag=1
        except: 
            flag=0
 
        if flag==1:
            #check whether parser generated subdirectories or not
            if os.path.exists('bulkparameters.csv')==False:
                ff = os.listdir(tmp_path)
                for checkfile in ff:
                    if os.path.isdir(checkfile)==True:
                
                        dumblk = pd.read_csv(os.path.join(tmp_path, checkfile,'bulkparameters.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
    
                        dumdisp = pd.read_csv(os.path.join(tmp_path, checkfile,'displacement.csv'),
                                      parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                
                        dumloc = pd.read_csv(os.path.join(tmp_path, checkfile, 'location.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser) 
                        duma1 = pd.read_csv(os.path.join(tmp_path,checkfile,'a1.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        duma2 = pd.read_csv(os.path.join(tmp_path,checkfile,'a2.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        dumb1 = pd.read_csv(os.path.join(tmp_path,checkfile,'b1.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        dumb2 = pd.read_csv(os.path.join(tmp_path,checkfile,'b2.csv'),
                                    parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        dumSxx = pd.read_csv(os.path.join(tmp_path,checkfile,'Sxx.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        dumSyy = pd.read_csv(os.path.join(tmp_path,checkfile,'Syy.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        dumSzz = pd.read_csv(os.path.join(tmp_path,checkfile,'Szz.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                        dumsst = pd.read_csv(os.path.join(tmp_path,checkfile,'sst.csv'),
                                     parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)                                                      
            else:
                dumblk = pd.read_csv('bulkparameters.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)          
                dumdisp = pd.read_csv('displacement.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)            
                dumloc = pd.read_csv('location.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)    
                duma1 = pd.read_csv('a1.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                duma2 = pd.read_csv('a2.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumb1 = pd.read_csv('b1.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumb2 = pd.read_csv('b2.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumSxx = pd.read_csv('Sxx.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumSyy = pd.read_csv('Syy.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumSzz = pd.read_csv('Szz.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)
                dumsst = pd.read_csv('sst.csv',parse_dates=[[0,1,2,3,4,5,6]],date_parser=sofar_date_parser)              
    
            if i == 0:
                bulkparams = dumblk
                displacements = dumdisp
                locations = dumloc
                a1 = duma1
                a2 = duma2
                b1 = dumb1
                b2 = dumb2
                Sxx = dumSxx
                Syy = dumSyy
                Szz = dumSzz
                sst = dumsst
            else:
                bulkparams = pd.concat([bulkparams,dumblk],ignore_index=True)
                displacements = pd.concat([displacements,dumdisp],ignore_index=True)
                locations = pd.concat([locations,dumloc],ignore_index=True)
                a1 = pd.concat([a1,duma1],ignore_index=True)
                a2 = pd.concat([a2,duma2],ignore_index=True)
                b1 = pd.concat([b1,dumb1],ignore_index=True)
                b2 = pd.concat([b2,dumb2],ignore_index=True)
                Sxx = pd.concat([Sxx,dumSxx],ignore_index=True)
                Syy = pd.concat([Syy,dumSyy],ignore_index=True)
                Szz = pd.concat([Szz,dumSzz],ignore_index=True)  
                sst = pd.concat([sst,dumsst],ignore_index=True)
      
            #delete files from tmp directory
            for i in os.listdir(tmp_path):
                if i[-3:]=='csv':
                    os.remove(i)
                elif i[-3:]=='CSV':
                    os.remove(i)
                elif os.path.isdir(i):
                    shutil.rmtree(i)    
        else:
            print('parser did not work')
    #delete tmp directory if last file      
    print('Finished processing ' + buoy)
    
    #export to csv
    #%% clean up for export
    os.chdir(outpath)
    shutil.rmtree(tmp_path)
    ############# BULKPARAMS ###############################################
    bulkparams = bulkparams.rename(columns={bulkparams.columns[0]:'time'})
    bulkparams = bulkparams.set_index('time').tz_localize(tz='UTC')
    bulkparams.to_csv(os.path.join(outpath,buoy+'_bulkparams.csv'))

    ########### DISPLACEMENTS ################################################
    displacements = displacements.rename(columns={displacements.columns[0]:'time'})
    displacements = displacements.set_index('time').tz_localize(tz='UTC')
    displacements.to_csv(os.path.join(outpath,buoy+'_displacements.csv'))

    ########### LOCATIONS ######################################################
    locations = locations.rename(columns={locations.columns[0]:'time'})
    locations = locations.set_index('time').tz_localize(tz='UTC')
    locations.to_csv(os.path.join(outpath,buoy+'_locations.csv'))

    ############ A1 ###########################################################
    a1 = a1.rename(columns={a1.columns[0]:'time'})
    a1 = a1.set_index('time').tz_localize(tz='UTC')
    a1.to_csv(os.path.join(outpath,buoy+'_a1.csv'))

    ############ A2 ###########################################################
    a2 = a2.rename(columns={a2.columns[0]:'time'})
    a2 = a2.set_index('time').tz_localize(tz='UTC')
    a2.to_csv(os.path.join(outpath,buoy+'_a2.csv'))

    ############ b1 ###########################################################
    b1 = b1.rename(columns={b1.columns[0]:'time'})
    b1 = b1.set_index('time').tz_localize(tz='UTC')
    b1.to_csv(os.path.join(outpath,buoy+'_b1.csv'))

    ############ b2 ###########################################################
    b2 = b2.rename(columns={b2.columns[0]:'time'})
    b2 = b2.set_index('time').tz_localize(tz='UTC')
    b2.to_csv(os.path.join(outpath,buoy+'_b2.csv'))

    ############ Sxx ###########################################################
    Sxx = Sxx.rename(columns={Sxx.columns[0]:'time'})
    Sxx = Sxx.set_index('time').tz_localize(tz='UTC')
    Sxx.to_csv(os.path.join(outpath,buoy+'_Sxx.csv'))

    ############ Syy ###########################################################
    Syy = Syy.rename(columns={Syy.columns[0]:'time'})
    Syy = Syy.set_index('time').tz_localize(tz='UTC')
    Syy.to_csv(os.path.join(outpath,buoy+'_Syy.csv'))

    ############ Szz ###########################################################
    Szz = Szz.rename(columns={Szz.columns[0]:'time'})
    Szz = Szz.set_index('time').tz_localize(tz='UTC')
    Szz.to_csv(os.path.join(outpath,buoy+'_Szz.csv'))

    ############ SST ###########################################################
    sst = sst.rename(columns={sst.columns[0]:'time'})
    sst = sst.set_index('time').tz_localize(tz='UTC')
    sst.to_csv(os.path.join(outpath,buoy+'_sst.csv'))
    print('done exporting processed data for ' + buoy)

outpath is P:\HANSEN_UWA-UNSW_Linkage\Data\CSIRO_Collab\DEP02_FEB-AUG\SPOT-1002
datapath isP:\HANSEN_UWA-UNSW_Linkage\Data\CSIRO_Collab\DEP02_FEB-AUG\SPOT-1002\raw
Concatenating all FLT files:
- 0288_FLT.CSV (File 1 out of 2)
- 0289_FLT.CSV (File 2 out of 2)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0288_SPC.CSV (File 1 out of 2)
- 0289_SPC.CSV (File 2 out of 2)
Processing spotter spectral output
Concatenating all SYS files:
- 0280_SYS.CSV (File 1 out of 10)
- 0281_SYS.CSV (File 2 out of 10)
- 0282_SYS.CSV (File 3 out of 10)
- 0283_SYS.CSV (File 4 out of 10)
- 0284_SYS.CSV (File 5 out of 10)
- 0285_SYS.CSV (File 6 out of 10)
- 0286_SYS.CSV (File 7 out of 10)
- 0287_SYS.CSV (File 8 out of 10)
- 0288_SYS.CSV (File 9 out of 10)
- 0289_SYS.CSV (File 10 out of 10)
Concatenating all LOC files:
- 0288_LOC.CSV (File 1 out of 2)
- 0289_LOC.CSV (File 2 out of 2)
Processing spotter displacement output
Concatenating all GPS f

Concatenating all FLT files:
- 0320_FLT.CSV (File 1 out of 10)
- 0321_FLT.CSV (File 2 out of 10)
- 0322_FLT.CSV (File 3 out of 10)
- 0323_FLT.CSV (File 4 out of 10)
- 0324_FLT.CSV (File 5 out of 10)
- 0325_FLT.CSV (File 6 out of 10)
- 0326_FLT.CSV (File 7 out of 10)
- 0327_FLT.CSV (File 8 out of 10)
- 0328_FLT.CSV (File 9 out of 10)
- 0329_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0320_SPC.CSV (File 1 out of 10)
- 0321_SPC.CSV (File 2 out of 10)
- 0322_SPC.CSV (File 3 out of 10)
- 0323_SPC.CSV (File 4 out of 10)
- 0324_SPC.CSV (File 5 out of 10)
- 0325_SPC.CSV (File 6 out of 10)
- 0326_SPC.CSV (File 7 out of 10)
- 0327_SPC.CSV (File 8 out of 10)
- 0328_SPC.CSV (File 9 out of 10)
- 0329_SPC.CSV (File 10 out of 10)
Processing spotter spectral output
Concatenating all SYS files:
- 0320_SYS.CSV (File 1 out of 10)
- 0321_SYS.CSV (File 2 out of 10)
- 0322_SYS.CSV (File 3 out of 10)
- 0323_SYS

- 0351_SST.CSV (File 2 out of 10)
- 0352_SST.CSV (File 3 out of 10)
- 0353_SST.CSV (File 4 out of 10)
- 0354_SST.CSV (File 5 out of 10)
- 0355_SST.CSV (File 6 out of 10)
- 0356_SST.CSV (File 7 out of 10)
- 0357_SST.CSV (File 8 out of 10)
- 0358_SST.CSV (File 9 out of 10)
- 0359_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0360_FLT.CSV (File 1 out of 10)
- 0361_FLT.CSV (File 2 out of 10)
- 0362_FLT.CSV (File 3 out of 10)
- 0363_FLT.CSV (File 4 out of 10)
- 0364_FLT.CSV (File 5 out of 10)
- 0365_FLT.CSV (File 6 out of 10)
- 0366_FLT.CSV (File 7 out of 10)
- 0367_FLT.CSV (File 8 out of 10)
- 0368_FLT.CSV (File 9 out of 10)
- 0369_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0360_SPC.CSV (File 1 out of 10)
- 0361_SPC.CSV (File 2 out of 10)
- 0362_SPC.CSV (File 3 out of 10)
- 0363_SPC.CSV (File 4 out of 10)
- 0364_SPC.CSV (File 5 out of 10)
-

Processing spotter displacement output
Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0390_SST.CSV (File 1 out of 10)
- 0391_SST.CSV (File 2 out of 10)
- 0392_SST.CSV (File 3 out of 10)
- 0393_SST.CSV (File 4 out of 10)
- 0394_SST.CSV (File 5 out of 10)
- 0395_SST.CSV (File 6 out of 10)
- 0396_SST.CSV (File 7 out of 10)
- 0397_SST.CSV (File 8 out of 10)
- 0398_SST.CSV (File 9 out of 10)
- 0399_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0400_FLT.CSV (File 1 out of 10)
- 0401_FLT.CSV (File 2 out of 10)
- 0402_FLT.CSV (File 3 out of 10)
- 0403_FLT.CSV (File 4 out of 10)
- 0404_FLT.CSV (File 5 out of 10)
- 0405_FLT.CSV (File 6 out of 10)
- 0406_FLT.CSV (File 7 out of 10)
- 0407_FLT.CSV (File 8 out of 10)
- 0408_FLT.CSV (File 9 out of 10)
- 0409_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0400_S

Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0430_SST.CSV (File 1 out of 10)
- 0431_SST.CSV (File 2 out of 10)
- 0432_SST.CSV (File 3 out of 10)
- 0433_SST.CSV (File 4 out of 10)
- 0434_SST.CSV (File 5 out of 10)
- 0435_SST.CSV (File 6 out of 10)
- 0436_SST.CSV (File 7 out of 10)
- 0437_SST.CSV (File 8 out of 10)
- 0438_SST.CSV (File 9 out of 10)
- 0439_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0440_FLT.CSV (File 1 out of 10)
- 0441_FLT.CSV (File 2 out of 10)
- 0442_FLT.CSV (File 3 out of 10)
- 0443_FLT.CSV (File 4 out of 10)
- 0444_FLT.CSV (File 5 out of 10)
- 0445_FLT.CSV (File 6 out of 10)
- 0446_FLT.CSV (File 7 out of 10)
- 0447_FLT.CSV (File 8 out of 10)
- 0448_FLT.CSV (File 9 out of 10)
- 0449_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0440_SPC.CSV (File 1 out of 10)
- 0441_SPC.CS

...

Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0387_SST.CSV (File 1 out of 10)
- 0388_SST.CSV (File 2 out of 10)
- 0389_SST.CSV (File 3 out of 10)
- 0390_SST.CSV (File 4 out of 10)
- 0391_SST.CSV (File 5 out of 10)
- 0392_SST.CSV (File 6 out of 10)
- 0393_SST.CSV (File 7 out of 10)
- 0394_SST.CSV (File 8 out of 10)
- 0395_SST.CSV (File 9 out of 10)
- 0396_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0397_FLT.CSV (File 1 out of 10)
- 0398_FLT.CSV (File 2 out of 10)
- 0399_FLT.CSV (File 3 out of 10)
- 0400_FLT.CSV (File 4 out of 10)
- 0401_FLT.CSV (File 5 out of 10)
- 0402_FLT.CSV (File 6 out of 10)
- 0403_FLT.CSV (File 7 out of 10)
- 0404_FLT.CSV (File 8 out of 10)
- 0405_FLT.CSV (File 9 out of 10)
- 0406_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0397_SPC.CSV (File 1 out of 10)
- 0398_SPC.CS

Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0427_SST.CSV (File 1 out of 10)
- 0428_SST.CSV (File 2 out of 10)
- 0429_SST.CSV (File 3 out of 10)
- 0430_SST.CSV (File 4 out of 10)
- 0431_SST.CSV (File 5 out of 10)
- 0432_SST.CSV (File 6 out of 10)
- 0433_SST.CSV (File 7 out of 10)
- 0434_SST.CSV (File 8 out of 10)
- 0435_SST.CSV (File 9 out of 10)
- 0436_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0437_FLT.CSV (File 1 out of 10)
- 0438_FLT.CSV (File 2 out of 10)
- 0439_FLT.CSV (File 3 out of 10)
- 0440_FLT.CSV (File 4 out of 10)
- 0441_FLT.CSV (File 5 out of 10)
- 0442_FLT.CSV (File 6 out of 10)
- 0443_FLT.CSV (File 7 out of 10)
- 0444_FLT.CSV (File 8 out of 10)
- 0445_FLT.CSV (File 9 out of 10)
- 0446_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0437_SPC.CSV (File 1 out of 10)
- 0438_SPC.CS

Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0467_SST.CSV (File 1 out of 10)
- 0468_SST.CSV (File 2 out of 10)
- 0469_SST.CSV (File 3 out of 10)
- 0470_SST.CSV (File 4 out of 10)
- 0471_SST.CSV (File 5 out of 10)
- 0472_SST.CSV (File 6 out of 10)
- 0473_SST.CSV (File 7 out of 10)
- 0474_SST.CSV (File 8 out of 10)
- 0475_SST.CSV (File 9 out of 10)
- 0476_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0477_FLT.CSV (File 1 out of 10)
- 0478_FLT.CSV (File 2 out of 10)
- 0479_FLT.CSV (File 3 out of 10)
- 0480_FLT.CSV (File 4 out of 10)
- 0481_FLT.CSV (File 5 out of 10)
- 0482_FLT.CSV (File 6 out of 10)
- 0483_FLT.CSV (File 7 out of 10)
- 0484_FLT.CSV (File 8 out of 10)
- 0485_FLT.CSV (File 9 out of 10)
- 0486_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0477_SPC.CSV (File 1 out of 10)
- 0478_SPC.CS

Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0507_SST.CSV (File 1 out of 10)
- 0508_SST.CSV (File 2 out of 10)
- 0509_SST.CSV (File 3 out of 10)
- 0510_SST.CSV (File 4 out of 10)
- 0511_SST.CSV (File 5 out of 10)
- 0512_SST.CSV (File 6 out of 10)
- 0513_SST.CSV (File 7 out of 10)
- 0514_SST.CSV (File 8 out of 10)
- 0515_SST.CSV (File 9 out of 10)
- 0516_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0517_FLT.CSV (File 1 out of 10)
- 0518_FLT.CSV (File 2 out of 10)
- 0519_FLT.CSV (File 3 out of 10)
- 0520_FLT.CSV (File 4 out of 10)
- 0521_FLT.CSV (File 5 out of 10)
- 0522_FLT.CSV (File 6 out of 10)
- 0523_FLT.CSV (File 7 out of 10)
- 0524_FLT.CSV (File 8 out of 10)
- 0525_FLT.CSV (File 9 out of 10)
- 0526_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Concatenating all SPC files:
- 0517_SPC.CSV (File 1 out of 10)
- 0518_SPC.CS

- 0556_LOC.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all GPS files:
  No _GPS data files available.
Concatenating all SST files:
- 0547_SST.CSV (File 1 out of 10)
- 0548_SST.CSV (File 2 out of 10)
- 0549_SST.CSV (File 3 out of 10)
- 0550_SST.CSV (File 4 out of 10)
- 0551_SST.CSV (File 5 out of 10)
- 0552_SST.CSV (File 6 out of 10)
- 0553_SST.CSV (File 7 out of 10)
- 0554_SST.CSV (File 8 out of 10)
- 0555_SST.CSV (File 9 out of 10)
- 0556_SST.CSV (File 10 out of 10)
Processing spotter displacement output
Concatenating all FLT files:
- 0557_FLT.CSV (File 1 out of 10)
- 0558_FLT.CSV (File 2 out of 10)
- 0559_FLT.CSV (File 3 out of 10)
- 0560_FLT.CSV (File 4 out of 10)
- 0561_FLT.CSV (File 5 out of 10)
- 0562_FLT.CSV (File 6 out of 10)
- 0563_FLT.CSV (File 7 out of 10)
- 0564_FLT.CSV (File 8 out of 10)
- 0565_FLT.CSV (File 9 out of 10)
- 0566_FLT.CSV (File 10 out of 10)
Processing spotter displacement output
- IIR phase correction using weight type:  0
Co